<a href="https://colab.research.google.com/github/abnerxch/bi-project/blob/main/An%C3%A1lisis_de_regresi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adriana Mundo, Ábner Xocop y Fernanda González

# Epidemiología social del SARS-CoV-2: análisis de regresión

## Prep env

In [ ]:
import pandas as pd
from google.colab import drive
from itertools import product
import numpy as np
from sklearn.linear_model import LinearRegression
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Merge exógenas y endógenas

In [ ]:
ZCTAs = pd.read_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/ZCTAs.csv')
CovidCases = pd.read_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/CovidCases.csv')

In [ ]:
StatesCensus = ZCTAs.groupby('State', as_index=False).agg(
    {
         'Insured': 'sum',
         'Uninsured': 'sum',
         'TotalPopulation': 'sum',
         'FemalePopulation': 'sum',
         'MalePopulation': 'sum',
         'White': 'sum',
         'AfricanAmerican': 'sum',
         'NativeAmerican': 'sum',
         'Asian': 'sum',
         'PacificIslander': 'sum',
         'Hispanic': 'sum',
         'Native': 'sum',
         'Naturalized': 'sum',
         'NonCitizen': 'sum',
         'GrandparentHouseholder': 'sum',
         'GrandparentPresence': 'sum',
         'FamilyHousehold': 'sum',
         'NonfamilyHousehold': 'sum',
         'MedianHouseholdIncome': 'mean',
         'CarTruckVan': 'sum',
         'PublicTransport': 'sum',
         'Bicycle': 'sum',
         'Walked': 'sum',
         'WFH': 'sum',
         'BelowPoverty': 'sum',
         'MedianEarnings': 'mean',
         'MedianEarningsMale': 'mean',
         'MedianEarningsFemale': 'mean',
         'MedianEarningsGenderInequality': 'mean',
         'EnrolledInSchool': 'sum',
         'EnrolledInHighschool': 'sum',
         'EnrolledInUndergrad': 'sum',
         'EnrolledInGrad': 'sum',
         'Employed': 'sum',
         'Unemployed': 'sum'
    }
)


In [ ]:
regression_analysis_data = pd.merge(CovidCases, StatesCensus, on="State")
regression_analysis_data.head()

,State,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio,Insured,Uninsured,TotalPopulation,FemalePopulation,MalePopulation,White,AfricanAmerican,NativeAmerican,Asian,PacificIslander,Hispanic,Native,Naturalized,NonCitizen,GrandparentHouseholder,GrandparentPresence,FamilyHousehold,NonfamilyHousehold,MedianHouseholdIncome,CarTruckVan,PublicTransport,Bicycle,Walked,WFH,BelowPoverty,MedianEarnings,MedianEarningsMale,MedianEarningsFemale,MedianEarningsGenderInequality,EnrolledInSchool,EnrolledInHighschool,EnrolledInUndergrad,EnrolledInGrad,Employed,Unemployed
0,AK,30555.0,176.0,3026.029499,0.606605,591927,121032,736703,352470,384233,539024,37816,143703,58362,13300,48819,681570,31077,24056,13194,16735,167411,84216,49475.753191,290790,5352,3535,26725,16003,72949,23191.885106,26427.778723,19118.029787,7309.748936,101963,41466,38404,10717,352511,30217
1,AL,79827.0,1529.0,5642.304949,2.134702,5986388,787285,6886256,3541236,3345020,4965735,1751063,85952,104741,7417,282057,6656277,80773,149206,165268,173692,1768286,859149,40646.393424,2696919,9661,2865,30511,80171,1231296,26787.485261,31863.710884,20864.679138,10999.031746,893155,364712,349633,67335,2886647,260141
2,AR,39378.0,660.0,5706.386278,1.813630,3920257,560373,4573624,2332108,2241516,3636078,760720,76254,69676,16522,328456,4360742,64949,147933,100957,113172,1170962,583660,37032.518750,1776542,7035,2920,32359,59849,847272,24404.956250,28486.857500,18622.853750,9864.003750,613897,240877,232716,38230,1925503,145741
3,AZ,544255.0,9950.0,5707.949725,2.860034,5017700,838618,5930470,2992657,2937813,4731307,302691,389161,230489,23747,1828317,5120205,322598,487667,131520,152179,1409878,787034,45044.634349,2196507,56048,26093,53868,140685,1089784,27716.426593,31524.498615,21996.202216,9528.296399,776576,323309,351632,71327,2550032,226240
4,CA,1237273.0,24101.0,3680.119957,1.307441,34389287,4991507,39884637,20072268,19812369,26130591,2742198,756235,6163509,310803,15555077,29153709,5238754,5492174,820467,1180119,9091351,4113590,61139.206935,14917863,898464,200275,460849,953423,6226936,32531.293065,38273.467002,25649.752796,12623.714206,5311301,2275100,2769513,521555,18114806,1744144


## Correlación

In [ ]:
regression_analysis_data.corr()

,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio,Insured,Uninsured,TotalPopulation,FemalePopulation,MalePopulation,White,AfricanAmerican,NativeAmerican,Asian,PacificIslander,Hispanic,Native,Naturalized,NonCitizen,GrandparentHouseholder,GrandparentPresence,FamilyHousehold,NonfamilyHousehold,MedianHouseholdIncome,CarTruckVan,PublicTransport,Bicycle,Walked,WFH,BelowPoverty,MedianEarnings,MedianEarningsMale,MedianEarningsFemale,MedianEarningsGenderInequality,EnrolledInSchool,EnrolledInHighschool,EnrolledInUndergrad,EnrolledInGrad,Employed,Unemployed
Confirmed,1.000000,0.830818,0.068721,0.016749,0.766597,0.715264,0.765189,0.762255,0.768079,0.722806,0.419266,0.646602,0.850498,0.504039,0.847722,0.704922,0.874049,0.869253,0.747072,0.798352,0.740714,0.725635,0.089147,0.741728,0.417015,0.870985,0.634736,0.812996,0.767048,0.073184,0.066775,0.084835,-0.000434,0.750986,0.772646,0.810760,0.774611,0.759847,0.817817
Deaths,0.830818,1.000000,-0.038759,0.164229,0.739685,0.587653,0.722772,0.723334,0.722083,0.669938,0.451987,0.543366,0.807286,0.404936,0.724473,0.662432,0.868328,0.805102,0.665295,0.719100,0.699477,0.719511,0.173195,0.649022,0.827017,0.797145,0.874521,0.728382,0.714481,0.159767,0.146986,0.170159,0.026512,0.691421,0.718265,0.760732,0.782678,0.726912,0.769002
IncidenceRate,0.068721,-0.038759,1.000000,0.036599,0.082905,0.143034,0.093404,0.093600,0.093188,0.108539,0.182242,0.131545,-0.112009,-0.345456,0.018411,0.122711,-0.047006,-0.007681,0.126708,0.085842,0.104823,0.094171,-0.324599,0.115483,-0.131057,-0.082225,-0.078256,0.046017,0.126611,-0.196432,-0.117898,-0.261210,0.185354,0.112972,0.096647,0.063063,0.029611,0.084038,0.062355
CaseFatalityRatio,0.016749,0.164229,0.036599,1.000000,0.127612,0.049998,0.117532,0.122288,0.112637,0.102475,0.274707,-0.125178,0.013354,-0.198145,0.006889,0.126599,0.083370,0.052420,0.094983,0.085672,0.121765,0.143104,0.420180,0.101241,0.260862,-0.014077,0.210174,0.050955,0.096440,0.525382,0.539063,0.490570,0.318345,0.109729,0.108406,0.097555,0.171101,0.125789,0.128946
Insured,0.766597,0.739685,0.082905,0.127612,1.000000,0.920099,0.998169,0.998338,0.997841,0.986829,0.786732,0.598608,0.805275,0.384463,0.851022,0.990440,0.862517,0.895681,0.958594,0.968759,0.997137,0.990853,0.033649,0.988266,0.548925,0.789505,0.831108,0.976866,0.981865,0.048473,0.057432,0.073947,-0.002000,0.988896,0.994180,0.991166,0.984784,0.998418,0.985749
Uninsured,0.715264,0.587653,0.143034,0.049998,0.920099,1.000000,0.942066,0.940562,0.943462,0.928934,0.784961,0.615323,0.715091,0.363851,0.903154,0.931693,0.791901,0.887628,0.977004,0.964838,0.937663,0.901217,-0.073479,0.949636,0.343624,0.671977,0.623202,0.922280,0.957437,-0.045454,-0.048411,-0.013469,-0.079259,0.958896,0.947736,0.924138,0.885906,0.931315,0.910054
TotalPopulation,0.765189,0.722772,0.093404,0.117532,0.998169,0.942066,1.000000,0.999926,0.999922,0.988458,0.795675,0.606598,0.798545,0.384482,0.866995,0.992097,0.859420,0.902822,0.971505,0.978070,0.998561,0.987634,0.016920,0.992836,0.522704,0.778155,0.807591,0.978372,0.988311,0.034192,0.041426,0.061039,-0.014199,0.994704,0.997392,0.990965,0.979860,0.998589,0.984050
FemalePopulation,0.762255,0.723334,0.093600,0.122288,0.998338,0.940562,0.999926,1.000000,0.999694,0.988120,0.801042,0.603032,0.795722,0.380802,0.862794,0.992485,0.858417,0.900388,0.970689,0.977042,0.998713,0.988714,0.017770,0.992197,0.527087,0.775332,0.809589,0.977404,0.988250,0.035905,0.042683,0.063693,-0.015958,0.993971,0.996792,0.990402,0.980606,0.998479,0.984443
MalePopulation,0.768079,0.722083,0.093188,0.112637,0.997841,0.943462,0.999922,0.999694,1.000000,0.988652,0.790048,0.610160,0.801316,0.388196,0.871169,0.991545,0.860316,0.905178,0.972191,0.978973,0.998251,0.986374,0.016045,0.993337,0.518127,0.780929,0.805417,0.979214,0.988220,0.032430,0.040131,0.058309,-0.012393,0.995302,0.997852,0.991389,0.978944,0.998546,0.983495
White,0.722806,0.669938,0.108539,0.102475,0.986829,0.928934,0.988458,0.988120,0.988652,1.000000,0.761761,0.578505,0.726928,0.326720,0.827504,0.9931

In [ ]:
correlation_analysis = regression_analysis_data.corr()
correlation_analysis.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/CorrelationAnalysis.csv', index = False)

## Regresión univariada

In [ ]:
# Y (endógena) | X (exógena) | R_sq (0-1)
Ys = ['Confirmed', 'Deaths', 'IncidenceRate',	'CaseFatalityRatio']
Xs = ['Insured', 'Uninsured', 'TotalPopulation', 'FemalePopulation', 'MalePopulation', 'White', 'AfricanAmerican', 'NativeAmerican', 'Asian', 'PacificIslander', 'Hispanic', 'Native', 'Naturalized', 'NonCitizen', 'GrandparentHouseholder', 'GrandparentPresence', 'FamilyHousehold', 'NonfamilyHousehold', 'MedianHouseholdIncome', 'CarTruckVan', 'PublicTransport', 'Bicycle', 'Walked', 'WFH', 'BelowPoverty', 'MedianEarnings', 'MedianEarningsMale', 'MedianEarningsFemale', 'MedianEarningsGenderInequality', 'EnrolledInSchool', 'EnrolledInHighschool', 'EnrolledInUndergrad', 'EnrolledInGrad', 'Employed', 'Unemployed']

regression_analysis_results = pd.DataFrame(columns = ['Y', 'X', 'R_sq'])
regression_analysis_results = pd.DataFrame(list(product(Ys, Xs)), columns=['Y', 'X'])
regression_analysis_results.head()

,Y,X
0,Confirmed,Insured
1,Confirmed,Uninsured
2,Confirmed,TotalPopulation
3,Confirmed,FemalePopulation
4,Confirmed,MalePopulation


In [ ]:
R_sq = []

for Y_name in Ys:
  Y = regression_analysis_data[Y_name]
  for X_name in Xs:
    X = regression_analysis_data[X_name].values.reshape((-1, 1))
    model = LinearRegression().fit(X, Y)
    r_sq = model.score(X, Y)
    R_sq.append(r_sq)

regression_analysis_results['R_sq'] = R_sq
regression_analysis_results.head()

,Y,X,R_sq
0,Confirmed,Insured,0.587671
1,Confirmed,Uninsured,0.511603
2,Confirmed,TotalPopulation,0.585514
3,Confirmed,FemalePopulation,0.581033
4,Confirmed,MalePopulation,0.589946


In [ ]:
regression_analysis_results.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/RegressionAnalysis.csv', index = False)